In [1]:
# устанавливаем нужную версию библиотеки sklearn, чтобы ответы сошлись
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd

В этом задании вы будете работать с [данными о пациентах, у части которых есть заболевание сердца](https://www.kaggle.com/datasets/redwankarimsony/heart-disease-data). Вам нужно построить модель, классифицирующую пациентов на больных этим заболеванием и тех, у кого его нет, а также проанализировать результаты.

In [3]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


1. Какой процент пациентов из представленных в данных имеет заболевание сердца (`'target' = 1`)? Ответ округлите до двух знаков после запятой.

In [4]:
# Подсчет общего числа пациентов и пациентов с заболеванием сердца
total_patients = len(data)
heart_disease_patients = data[data['target'] == 1].shape[0]

# Вычисление процента
percentage = (heart_disease_patients / total_patients) * 100
print(f"Процент пациентов с заболеванием сердца: {percentage:.2f}%")


Процент пациентов с заболеванием сердца: 54.46%


Разделим данную выборку на обучающую и тестовую части в отношении 3:1.

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.25, random_state=13)
X_train.shape, X_test.shape

((227, 13), (76, 13))

2. Обучите линейный классификатор из `sklearn` (`sklearn.linear_model.SGDClassifier`) с максимальным числом итераций `max_iter=1000`, постоянным значением шага градиентного спуска (`learning_rate='constant'`), равным `eta0=0.1`. В качестве сида поставьте `random_state=13`. Параметр регуляризации отключите: `alpha=0`. Класс `sklearn.linear_model.SGDClassifier` объединяет в себе разные линейные модели - чтобы получить логистическую регрессию, зафиксируйте параметр `loss='log'`.

   Подробнее о параметрах можно почитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html).

   Какое значение свободного коэффициента у полученного линейного классификатора? Ответ округлите до двух знаков после запятой.

In [6]:
from sklearn.linear_model import SGDClassifier

# Создание и настройка классификатора
clf = SGDClassifier(max_iter=1000, learning_rate='constant', eta0=0.1, alpha=0, loss='log_loss', random_state=13)

# Обучение модели на обучающей выборке
clf.fit(X_train, y_train)

# Получение значения свободного коэффициента (intercept)
intercept = clf.intercept_[0]
print(f"Свободный коэффициент: {intercept:.2f}")

Свободный коэффициент: -1.16


3. Какое значение L2-нормы вектора весов (без учета свободного коэффициента) у полученного линейного классификатора? Ответ округлите до двух знаков после запятой.

_Напоминание. L2-норма вектора $v = (v_1, \ldots, v_n)$ - это корень из суммы квадратов его элементов:_

$$
\|v\|_2 = \sqrt{\sum\limits_{i=1}^nv_i^2}
$$

In [7]:
# Получение весов модели (без учета свободного коэффициента)
weights = clf.coef_

# Вычисление L2-нормы
l2_norm = np.sqrt(np.sum(weights**2))
l2_norm_rounded = round(l2_norm, 2)
print(f"L2-норма вектора весов: {l2_norm_rounded}")

L2-норма вектора весов: 175.68


4. Найдите долю правильных ответов классификатора на тестовой части выборки **(в процентах)**. Ответ округлите до двух знаков после запятой. Например, если значение доли правильных ответов будет равно 0.1234, то ответом будет 12.34 - ведь это 12.34%.

In [8]:
# Получение доли правильных ответов
accuracy = clf.score(X_test, y_test)

# Перевод в проценты и округление
accuracy_percentage = round(accuracy * 100, 2)
print(f"Доля правильных ответов: {accuracy_percentage:.2f}%")

Доля правильных ответов: 61.84%


5. В задаче классификации, как и в задаче регрессии, для оптимизации линейных моделей можно применять регуляризацию. Этот метод реализован и в `sklearn.linear_model.SGDClassifier` - параметр регуляризации обозначается параметром `alpha`. За тип регуляризации (L1, L2 или обе сразу) отвечает параметр `penalty`.

   Обучите классификатор заново с параметром регуляризации `alpha=0.1` и типом регуляризации `penalty='l1'`. Оставьте максимальное число итераций, равное `max_iter=1000` и сид `random_state=13`. Также вместо постоянного значения шага градиентного спуска используйте оптимальное (`learning_rate='optimal'`), которое, кстати, зависит от `alpha` (о том, как именно он вычисляется и какие еще параметры можно выбрать, можно подробнее прочитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)). В данном случае значение начального шага градиентного спуска `eta0` никак не участвует в обучении.
   
   Отличается ли качество полученного классификатора от качества первого? Какая доля правильных ответов получается теперь на тестовой выборке? Выразите ее **в процентах**, ответ округлите до двух знаков после запятой. Например, если значение доли правильных ответов будет равно 0.1234, то ответом будет 12.34 - ведь это 12.34%.

In [9]:
# Создание и настройка нового классификатора с L1-регуляризацией
clf_l1 = SGDClassifier(max_iter=1000, alpha=0.1, penalty='l1', learning_rate='optimal', loss='log_loss', random_state=13)

# Обучение модели на обучающей выборке
clf_l1.fit(X_train, y_train)

# Получение доли правильных ответов на тестовой выборке
accuracy_l1 = clf_l1.score(X_test, y_test)

# Перевод в проценты и округление
accuracy_l1_percentage = round(accuracy_l1 * 100, 2)
print(f"Доля правильных ответов с L1-регуляризацией: {accuracy_l1_percentage:.2f}%")

Доля правильных ответов с L1-регуляризацией: 76.32%


6. Найдите L2-норму вектора весов для полученного классификатора (заметьте, как на нее повлияла регуляризация). Ответ округлите до двух знаков после запятой.

    Заметьте, что вектор стал более разреженным, и в нем появились нулевые элементы - это результат действия L1-регуляризации.

In [10]:
# Получение весов модели с L1-регуляризацией (без учета свободного коэффициента)
weights_l1 = clf_l1.coef_

# Вычисление L2-нормы
l2_norm_l1 = np.sqrt(np.sum(weights_l1**2))
l2_norm_l1_rounded = round(l2_norm_l1, 2)
print(f"L2-норма вектора весов с L1-регуляризацией: {l2_norm_l1_rounded}")

L2-норма вектора весов с L1-регуляризацией: 10.43


7. Наконец, проверьте, как полученные классификаторы предсказывают не классы, а вероятности классов - так как мы работаем с логистической регрессией, это можно сделать. Посмотрите на вероятности, которые выдает первый классификатор (обученный с постоянным шагом градиентного спуска и без регуляризации) на тестовой части выборки. В этом вам поможет метод `predict_proba`. Результатом его работы будет список размера $N\times 2$, где $N$ - это число объектов. В каждом столбце списка находятся вероятности соответствующего класса для объектов. Поэтому если вам нужен положительный класс, вас интересует последний столбец.

    Какое получается значение AUC-ROC? Ответ округлите до двух знаков после запятой.

In [11]:
from sklearn.metrics import roc_auc_score

# Получение вероятностей классов для тестовой выборки
probabilities = clf.predict_proba(X_test)

# Вероятности положительного класса
positive_class_probabilities = probabilities[:, 1]

# Вычисление AUC-ROC
auc_roc = roc_auc_score(y_test, positive_class_probabilities)

# Округление результата до двух знаков
auc_roc_rounded = round(auc_roc, 2)
print(f"AUC-ROC: {auc_roc_rounded}")

AUC-ROC: 0.57


8. Посмотрите на вероятности, которые выдает второй классификатор (обученный с оптимальным шагом градиентного спуска и с регуляризацией) на тестовой части выборки. Что вы наблюдаете - как отличаются эти вероятности от вероятностей первого классификатора?

   Посчитайте значение AUC-ROC второго классификатора. Ответ округлите до двух знаков после запятой.

In [12]:
from sklearn.metrics import roc_auc_score

# Получение вероятностей классов для тестовой выборки с вторым классификатором (с регуляризацией)
probabilities_l1 = clf_l1.predict_proba(X_test)

# Вероятности положительного класса
positive_class_probabilities_l1 = probabilities_l1[:, 1]

# Вычисление AUC-ROC для второго классификатора
auc_roc_l1 = roc_auc_score(y_test, positive_class_probabilities_l1)

# Округление результата до двух знаков
auc_roc_l1_rounded = round(auc_roc_l1, 2)
print(f"AUC-ROC для классификатора с L1-регуляризацией: {auc_roc_l1_rounded}")


AUC-ROC для классификатора с L1-регуляризацией: 0.86


9. Какой признак является самым важным по мнению лучшей модели (имеет наибольший по модулю коэффициент) для принятия решения?

In [13]:
# Получение весов модели с L1-регуляризацией
feature_importances = clf_l1.coef_[0]

# Находим индекс самого важного признака
most_important_feature_index = np.argmax(np.abs(feature_importances))

# Определяем название признака
most_important_feature_name = X_train.columns[most_important_feature_index]
most_important_feature_value = feature_importances[most_important_feature_index]

print(f"Самый важный признак: {most_important_feature_name}")
print(f"Его значение (вес): {most_important_feature_value:.2f}")


Самый важный признак: cp
Его значение (вес): 9.26
